In [1]:
import tensorflow
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout , Activation, Flatten , Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import SGD
import random
import json
import pickle

In [2]:
words=[]
labels = []
docs = []
ignore_list = ['?', '!']

In [3]:
dataset = open('/content/intents.json').read()
intents = json.loads(dataset)

In [4]:
import nltk

nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        word_token = nltk.word_tokenize(pattern)
        words.extend(word_token)
        #add documents in the corpus
        docs.append((word_token, intent['tag']))

        # add to our labels list
        if intent['tag'] not in labels:
            labels.append(intent['tag'])

In [6]:
import nltk

nltk.download('wordnet')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
# lemmatize each word, and sort words by removing duplicates:
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in ignore_list]
words = sorted(list(set(words)))
# sort labels:
labels = sorted(list(set(labels)))

In [8]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(labels,open('labels.pkl','wb'))

In [9]:
# creating our training data:
training_data = []
# creating an empty array for our output (with size same as length of labels):
output = [0]*len(labels)

for doc in docs:
    bag_of_words = []
    pattern_words = doc[0]
    #lemmatize pattern words:
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]

    for w in words:
        if w in pattern_words:
            bag_of_words.append(1)
        else:
            bag_of_words.append(0)

    output_row = list(output)
    output_row[labels.index(doc[1])] = 1

    training_data.append([bag_of_words,output_row])

In [10]:
# convert training_data to numpy array and shuffle the data:
random.shuffle(training_data)
training_data = np.array(training_data)

<ipython-input-10-d2a7c1a73fe0>:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training_data = np.array(training_data)


In [11]:
# Now we have to create training list:
x_train = list(training_data[:,0])
y_train = list(training_data[:,1])

In [12]:
# Creating Model:

model = Sequential()
model.add(Dense(128, input_shape=(len(x_train[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(y_train[0]), activation='softmax'))

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               16768     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 30)                1950      
                                                                 
Total params: 26974 (105.37 KB)
Trainable params: 26974 (105.37 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [14]:
import tensorflow as tf

# Create the SGD optimizer without the decay parameter
sgd_optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)

# Compile your model
model.compile(loss='categorical_crossentropy', optimizer=sgd_optimizer, metrics=['accuracy'])


In [15]:
# fit the model
history = model.fit(np.array(x_train), np.array(y_train), epochs=200, batch_size=5, verbose=1)

Epoch 1/200
23/23 [==============================] - 1s 3ms/step - loss: 3.4167 - accuracy: 0.0354
Epoch 2/200
23/23 [==============================] - 0s 3ms/step - loss: 3.3213 - accuracy: 0.0885
Epoch 3/200
23/23 [==============================] - 0s 2ms/step - loss: 3.3098 - accuracy: 0.1327
Epoch 4/200
23/23 [==============================] - 0s 3ms/step - loss: 3.2586 - accuracy: 0.0973
Epoch 5/200
23/23 [==============================] - 0s 2ms/step - loss: 3.1624 - accuracy: 0.1416
Epoch 6/200
23/23 [==============================] - 0s 2ms/step - loss: 3.0764 - accuracy: 0.1947
Epoch 7/200
23/23 [==============================] - 0s 3ms/step - loss: 3.0164 - accuracy: 0.1770
Epoch 8/200
23/23 [==============================] - 0s 2ms/step - loss: 2.9159 - accuracy: 0.2478
Epoch 9/200
23/23 [==============================] - 0s 2ms/step - loss: 2.8559 - accuracy: 0.2478
Epoch 10/200
23/23 [==============================] - 0s 2ms/step - loss: 2.7487 - accuracy: 0.2389
Epoch 11/

In [16]:
model.save('ChatBotModel', history)

In [17]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np
import json
import random
from keras.models import load_model

model = load_model('ChatBotModel')

intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
labels = pickle.load(open('labels.pkl','rb'))

In [18]:
def bank_of_words(s,words, show_details=True):
    bag_of_words = [0 for _ in range(len(words))]
    sent_words = nltk.word_tokenize(s)
    sent_words = [lemmatizer.lemmatize(word.lower()) for word in sent_words]
    for sent in sent_words:
        for i,w in enumerate(words):
            if w == sent:
                bag_of_words[i] = 1
    return np.array(bag_of_words)

def predict_label(s, model):
    # filtering out predictions
    pred = bank_of_words(s, words,show_details=False)
    response = model.predict(np.array([pred]))[0]
    ERROR_THRESHOLD = 0.25
    final_results = [[i,r] for i,r in enumerate(response) if r>ERROR_THRESHOLD]
    final_results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in final_results:
        return_list.append({"intent": labels[r[0]], "probability": str(r[1])})
    return return_list

In [19]:
def Response(ints, intents_json):
    tags = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tags):
            response = random.choice(i['responses'])
            break
    return response

def chatbot_response(msg):
    ints = predict_label(msg, model)
    response = Response(ints, intents)
    return response

In [20]:
def chat():
    print("Start chat with ChatBot of Deepu")
    while True:
        inp = input("You: ")
        if inp.lower() == 'quit':
            break
        response = chatbot_response(inp)
        print("\n BOT: " + response + '\n\n')

chat()

Start chat with ChatBot of Deepu
You: Hii
1/1 [==============================] - 0s 143ms/step

 BOT: Redirecting to Google...


You: Hi
1/1 [==============================] - 0s 24ms/step

 BOT: Hello


You: jfakdfnidnhf
1/1 [==============================] - 0s 24ms/step

 BOT: Redirecting to Google...


You: quit
